In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

import sys
sys.path.append("/home/z5297792/UNSW-MRes/MRes/modules") 
from utils import dopioe, rossby_number, calc_tang_vel, find_directional_radii



In [2]:
import netCDF4 as nc
from scipy.interpolate import griddata
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import time

# Field Data

fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'

dataset = nc.Dataset(fname)

lon_rho = np.transpose(dataset.variables['lon_rho'], axes=(1, 0))
lat_rho = np.transpose(dataset.variables['lat_rho'], axes=(1, 0))
mask_rho = np.transpose(dataset.variables['mask_rho'], axes=(1, 0))
h =  np.transpose(dataset.variables['h'], axes=(1, 0))
angle = dataset.variables['angle'][0, 0]
z_r = np.load('/srv/scratch/z5297792/z_r.npy')
z_r = np.transpose(z_r, (1, 2, 0))[150, 150, :]

def distance(lat1, lon1, lat2, lon2):
    EARTH_RADIUS = 6357
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return EARTH_RADIUS * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

j_mid = lon_rho.shape[1] // 2
i_mid = lon_rho.shape[0] // 2

dx = distance(lat_rho[:-1, j_mid], lon_rho[:-1, j_mid],
              lat_rho[1:, j_mid], lon_rho[1:, j_mid])
dy = distance(lat_rho[i_mid, :-1], lon_rho[i_mid, :-1],
              lat_rho[i_mid, 1:], lon_rho[i_mid, 1:])

x_grid = np.insert(np.cumsum(dx), 0, 0)
y_grid = np.insert(np.cumsum(dy), 0, 0)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid, indexing='ij')

res = 1  # 1 km resolution
x_new = np.arange(0, x_grid[-1], res)
y_new = np.arange(0, y_grid[-1], res)
X_new, Y_new = np.meshgrid(x_new, y_new, indexing='ij')
new_points = np.column_stack((X_new.ravel(), Y_new.ravel()))

interp_lon = RegularGridInterpolator((x_grid, y_grid), lon_rho,
                                     method='linear', bounds_error=False, fill_value=np.nan)
interp_lat = RegularGridInterpolator((x_grid, y_grid), lat_rho,
                                     method='linear', bounds_error=False, fill_value=np.nan)

lon_new = interp_lon(new_points).reshape(len(x_new), len(y_new))
lat_new = interp_lat(new_points).reshape(len(x_new), len(y_new))


In [3]:
df_eddies = pd.read_pickle('/srv/scratch/z5297792/Clim_data/df_eddies_processed.pkl')


In [4]:
sample_eddies = [8, 896, 2504, 2749, 382, 926, 1394, 1967]

In [5]:
def compute_vert_data(df_eddies, eddy, X, Y, z_r, r=30):

    df = df_eddies[df_eddies['ID'] == eddy].copy()
    dic_data = {f'Eddy{eddy}': {}}
    fnumber_old = None
    x, y = X[:,0], Y[0,:]
    timer = 0

    # DOPIOE wont work if too close to boundary
    x = X[:, 0]
    y = Y[0, :]
    dx = np.max(np.diff(x))  # spacing in x-direction
    dy = np.max(np.diff(y))  # spacing in y-direction
    cell_size = np.max([dx, dy])        # average cell size in Euclidean units
    margin = int(np.ceil(r / cell_size))  

    for t, data in df.iterrows():
        day, fname, xc_surf, yc_surf, w_surf = data['Day'], data['fname'], data['x0'], data['y0'], data['w']
        fnumber = fname[-8:-3]

        # Only load if file changes
        if fnumber != fnumber_old:
            u_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/u_{fnumber}.npy')
            v_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/v_{fnumber}.npy')
            u_raw = np.where(np.abs(u_raw) > 1e30, np.nan, u_raw).astype(float)
            v_raw = np.where(np.abs(v_raw) > 1e30, np.nan, v_raw).astype(float)
            with nc.Dataset(fname) as dataset:
                ocean_time = dataset.variables['ocean_time'][:].data / 86400
            fnumber_old = fnumber
        t_rel = np.where(ocean_time == day)[0][0]

        u_t, v_t = u_raw[:, :, :, t_rel], v_raw[:, :, :, t_rel]

        df_data = []
        xc, yc = xc_surf, yc_surf
        xc_old, yc_old = xc_surf, yc_surf

        # for k in range(24):
        for k in range(u_t.shape[-1]):
            u, v = u_t[:, :, k], v_t[:, :, k]

            R_grid = np.hypot(xc - X, yc - Y)
            ic, jc = map(int, np.unravel_index(np.argmin(R_grid), R_grid.shape))
            
            if (ic < margin or ic >= X_grid.shape[0] - margin or
                jc < margin or jc >= X_grid.shape[1] - margin):
                break

            # horizontal transect (constant y = y[jc])
            x_mask = np.abs(x - xc) < r
            x1 = x[x_mask]
            y1 = np.full_like(x1, y[jc])
            u1 = u[x_mask, jc]
            v1 = v[x_mask, jc]
        
            # vertical transect (constant x = x[ic])
            y_mask = np.abs(y - yc) < r
            y2 = y[y_mask]
            x2 = np.full_like(y2, x[ic])
            u2 = u[ic, y_mask]
            v2 = v[ic, y_mask]

            xc, yc, w, Q, _, psi0, _ = dopioe(x1, y1, u1, v1, x2, y2, u2, v2)

            if (np.sign(w) != np.sign(w_surf)) or (np.hypot(xc_old - xc, yc_old - yc) > 50):
                break
            else:
                xc_old, yc_old = xc, yc

            w *= 1e-3 # to s^-1
            Ro = rossby_number(w, yc)

            radii = find_directional_radii(u, v, X, Y, xc, yc, calc_tang_vel)
            Rc = np.mean([radii['up'], radii['right'], radii['down'], radii['left']])

            df_data.append({
                'x': xc, 'y': yc, 'Q': Q, 'w': w,
                'Rc': Rc, 'psi0': psi0, 'Ro': Ro,
                'z': k, 'Depth': z_r[k]
            })

        if df_data:
            df_data = pd.DataFrame(df_data)
            dx = df_data['x'].diff()
            dy = df_data['y'].diff()
            df_data['TD'] = np.hypot(df_data['x'] - df_data.iloc[0]['x'],
                                  df_data['y'] - df_data.iloc[0]['y'])
            dic_data[f'Eddy{eddy}'][f'Day{day}'] = df_data
        else:
            dic_data[f'Eddy{eddy}'][f'Day{day}'] = pd.DataFrame(columns=['x', 'y', 'Q', 'w', 'Rc', 'psi0', 'Ro', 'z','Depth'])

        # if timer % 10 == 0:
        #     print(f'{timer / len(df) * 100:.1f}%')
        #     timer += 1
        
    return dic_data
    

In [6]:
dic_sample = {}
for e, eddy in enumerate(sample_eddies):
    dic = compute_vert_data(df_eddies, eddy, X_grid, Y_grid, z_r)
    dic_sample = dic_sample | dic 
    print(f'{e+1}/{len(sample_eddies)}')
    

/home/z5297792/UNSW-MRes/MRes/modules/utils.py:372: RuntimeWarning: overflow encountered in exp
  E = np.exp(gamma / psi0)
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:375: RuntimeWarning: overflow encountered in square
  return np.sum((ui - u_pred)**2 + (vi - v_pred)**2)
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:375: RuntimeWarning: overflow encountered in square
  return np.sum((ui - u_pred)**2 + (vi - v_pred)**2)
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:375: RuntimeWarning: overflow encountered in square
  return np.sum((ui - u_pred)**2 + (vi - v_pred)**2)
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:375: RuntimeWarning: overflow encountered in square
  return np.sum((ui - u_pred)**2 + (vi - v_pred)**2)


1/8


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:375: RuntimeWarning: overflow encountered in square
  return np.sum((ui - u_pred)**2 + (vi - v_pred)**2)


2/8


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:375: RuntimeWarning: overflow encountered in square
  return np.sum((ui - u_pred)**2 + (vi - v_pred)**2)
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:372: RuntimeWarning: overflow encountered in exp
  E = np.exp(gamma / psi0)


3/8


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:375: RuntimeWarning: overflow encountered in square
  return np.sum((ui - u_pred)**2 + (vi - v_pred)**2)
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:375: RuntimeWarning: overflow encountered in square
  return np.sum((ui - u_pred)**2 + (vi - v_pred)**2)
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:372: RuntimeWarning: overflow encountered in exp
  E = np.exp(gamma / psi0)


4/8
5/8


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:375: RuntimeWarning: overflow encountered in square
  return np.sum((ui - u_pred)**2 + (vi - v_pred)**2)


6/8
7/8
8/8


In [8]:
with open('/srv/scratch/z5297792/Clim_data/Sample_Data/dic_sample_vert_info.pkl', 'wb') as f:
    pickle.dump(dic_sample, f)
    